In [30]:
import optuna
from config import quality_metrics, layout, dataset, paths
import pandas as pd
import plotly.express as px


In [33]:
UUID = "25a73bb2-ae88-4b4a-9f14-0acf4ee3b624"
P_NAMES = ["number_of_pivots", "number_of_iterations", "eps"]
NAME_ABBREVIATIONS = {
    "angular_resolution": "ANR",
    "aspect_ratio": "AR",
    "crossing_angle": "CA",
    "crossing_number": "CN",
    "gabriel_graph_property": "GB",
    "ideal_edge_lengths": "IE",
    "node_resolution": "NR",
    "run_time": "RT",
    "neighborhood_preservation": "NP",
    "stress": "ST",
}

for layout_name in layout.LAYOUT_NAMES:
    for dataset_name in dataset.DATASET_NAMES:
        # if dataset_name != "USpowerGrid":
        #     continue
        data_dir = paths.get_data_dir(
            layout_name=layout_name, dataset_name=dataset_name, uuid=UUID
        )
        if not data_dir.exists():
            continue

        study = optuna.load_study(
            storage=f"sqlite:///{data_dir.joinpath('optimization').joinpath('100trials-fixed_seed.sql').resolve()}",
            study_name="angular_resolution,aspect_ratio,crossing_angle,crossing_number,gabriel_graph_property,ideal_edge_lengths,neighborhood_preservation,node_resolution,stress",
        )
        print(dataset_name, len(study.trials), len(study.best_trials))
        qualities_list = []
        for best_trial in study.best_trials:
            qualities = {}
            for qm_name in quality_metrics.ALL_QM_NAMES:
                qualities[NAME_ABBREVIATIONS[qm_name]] = best_trial.user_attrs[
                    "qualities"
                ][qm_name]
            qualities_list.append(qualities)

        df = pd.DataFrame(qualities_list)
        fig = px.scatter_matrix(
            df,
            title=f"{dataset_name} {len(study.best_trials)}best_trials/{len(study.trials)}trials",
            dimensions=[
                NAME_ABBREVIATIONS[qm_name]
                for qm_name in quality_metrics.ALL_QM_NAMES
            ],
        )
        fig.update_traces(marker=dict(size=2))
        fig.update_layout(
            margin=dict(t=30, r=20, b=10),
        )
        fig.show()

        # fig.write_image(f"images/{dataset_name}/{qm_name}.png")

1138_bus 1000 609


USpowerGrid 1000 711


les_miserables 1000 593
